In [3]:
import os
from pathlib import Path
import math
import json
import numpy as np
import torch
import time
import matplotlib.pyplot as plt
import imp
from toolbox import losses
imp.reload(losses)
from toolbox.losses import coloring_loss, triplet_loss
from toolbox import metrics
from loaders.data_generator import KCOL_Generator
from toolbox.metrics import all_losses_acc, accuracy_linear_assignment
from toolbox.utils import check_dir
from models import coloring_model, get_siamese_model_test
imp.reload(coloring_model)

cwd = os.getcwd()

In [2]:
def get_device_config(model_path):
    """ Get the same device as used for training """
    config_file = os.path.join(model_path,'config.json')
    with open(config_file) as json_file:
        config_model = json.load(json_file)
    use_cuda = not config_model['cpu'] and torch.cuda.is_available()
    device = 'cuda' if use_cuda else 'cpu'
    return config_model, device

def creat_args(config, num_ex = 1000):
    args = config['data']
    args['num_examples_test'] = num_ex
    n_vertices = args['n_vertices']
    edge_density = args['edge_density']
    deg = (n_vertices)*edge_density
    print(f'graphs with {n_vertices} vertices and average degree {deg}')
    return args, deg

def acc_2_error(mean_acc, q_acc):
    error = q_acc-mean_acc[:,np.newaxis]
    error[:,0] = -error[:,0]
    return error

def compute_dataset(args,path_dataset,train=True,bs=10):
    num_batches = math.ceil(args['num_examples_val']/bs)
    if train:
        gene = GCP_Generator('train', args, path_dataset)
    else:
        gene = GCP_Generator('test', args, path_dataset)
    gene.load_dataset()
    loader = siamese_loader(gene, bs, gene.constant_n_vertices)
    return loader

def compute_quant(all_acc,quant_low=0.1,quant_up=0.9):
    mean_acc = np.mean(all_acc,1)
    num = len(mean_acc)
    q_acc = np.zeros((num,2))
    for i in range(num):
        q_acc[i,:] = np.quantile(all_acc[i,:],[quant_up, quant_low])
    return mean_acc, q_acc

def train_epoch(model, embed_model, train_loader):
    loss_fn = coloring_loss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    model.train()
    cum_loss = 0
    for idx, (graph,tgt) in enumerate(train_loader):
        graph = graph.to(device)
        embed = embed_model(graph)
        
        tgt = tgt.to(device)
        
        out = model(embed)
        
        optimizer.zero_grad()

        loss = loss_fn(graph, out, tgt)
        loss.backward()

        optimizer.step()
        cum_loss += loss.item()
    return cum_loss / len(train_loader)


def evaluate(model, val_loader):
    model.eval()
    cum_loss = 0
    for idx, (graph, tgt) in (enumerate(valid_loader)):
        graph = graph.to(device)
        embed = embed_model(graph)
        
        tgt = tgt.to(device)
        
        out = model(embed)
        
        loss = loss_fn(out, tgt)
        cum_loss += loss.item()
    return cum_loss / len(val_loader) 
    

## Loading the pretrained model

In [2]:
model_path = cwd+'experiments-gnn/qap/expe_new/node_embedding_rec_regular_150_0.05/05-15-23-16-14'
config_model, device = get_device_config(model_path)
embed_model = get_siamese_model_test(config_model["test"]["path_model"])
embed_model.to(device)
criterion = triplet_loss()

NameError: name 'cwd' is not defined

In [25]:
args,deg  = creat_args(config_model)

graphs with 50 vertices and average degree 10.0


## Generating data and loaders

In [3]:
args['num_colors_low'] = 3
args['num_colors_high'] = 5
args['num_examples_train'] = 200
args['num_examples_val'] = 10
train_loader = compute_dataset(args, "downloads")
valid_loader = compute_dataset(args, "downloads", train=False)

NameError: name 'args' is not defined

## Training the coloring model

In [4]:
model = coloring_model.ColoringModel(args['n_vertices'],embed_dim=64)
model.to(device)

NameError: name 'args' is not defined

In [28]:
num_epochs = 30
for epoch in range(1, num_epochs+1):
    start_time = time.time()
    train_loss = train_epoch(model, embed_model.node_embedder, train_loader)
    end_time = time.time()
    val_loss = evaluate(model, valid_loader)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "
          f"Epoch time = {(end_time - start_time):.3f}s"))

NameError: name 'valid_loader' is not defined